In [1]:
import scanpy as sc
import numpy as np
import torch
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append('/data/nelkazwi/code/cisTopic/')
from cisTopic import cisTopic


In [13]:
path = '/data/nelkazwi/RNA_velo/Unitvelo_atac/HSPC/'
adata_atac = sc.read(path+"ATAC_data.h5ad")

In [5]:
n_samples = 3000
n_burnin = 10
T = 30
n_b_samples = int(n_samples/n_burnin)
batch_size = 500
alpha = 50/T
beta = 0.1


cis_t = cisTopic(adata_atac, T, alpha, beta)

In [ ]:
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'
print(f'Running on: {device}')

theta, phi = cis_t.fit(batch_size, n_samples, n_burnin, dev=device, save_data=False)

In [12]:

path_samples = '/scratch/nelkazwi/RNA_velo/Unitvelo_atac/HSPC/30_topics/'

theta=torch.load(path_samples+"theta_ATAC_"+str(T)+".txt",map_location=torch.device('cpu'))
m=theta[50:,:,:].mean(axis=0)
m=m/m.sum(axis=1)[:,np.newaxis] 

phi = torch.load(path_samples+"phi_ATAC_"+str(T)+".txt",map_location=torch.device('cpu'))
m_phi = phi[50:,:,:].mean(axis=0)
m_phi =m_phi /m_phi.sum(axis=1)[:,np.newaxis]

In [16]:
p = torch.matmul(m,m_phi)
adata_atac.obsm["cisTopic"] = p.numpy()
adata_atac.write_h5ad("UniTVelo-ATAC/notebooks/data/HSPC_dataset/HSPC_adata_atac.h5ad")